In [75]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from network import network

In [76]:
# Step 1: Load and preprocess the data
data = pd.read_csv('diabetes_clean_2015.csv')
X = data.iloc[:, 1:].values  # Input features
y = data.iloc[:, 0].values    # Target labels

In [77]:
#Step: 2
# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)  # Add an extra dimension for the target

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [78]:
# Step 3: Move the model and data to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
model = network().to(device)
X_train, y_train = X_train.to(device), y_train.to(device)

Device: cpu


In [79]:
def train_model(model, train_loader, criterion, optimizer, device, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        for inputs, targets in train_loader:
            # Move inputs and targets to device
            inputs, targets = inputs.to(device), targets.to(device) 
            
            # Ensure inputs and targets have the correct dtype and shape
            print(f"Inputs: {inputs.dtype}, {inputs.shape}")
            print(f"Targets: {targets.dtype}, {targets.shape}")
            
            # Zero the gradients
            optimizer.zero_grad()
            
            try:
                # Forward pass
                outputs = model(inputs)
                
                # Ensure outputs have the correct dtype and shape
                print(f"Outputs: {outputs.dtype}, {outputs.shape}")
                
                # Calculate loss
                loss = criterion(outputs, targets)
                
                # Backward pass
                loss.backward()
                
                # Update weights
                optimizer.step()
                
                print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')
            
            except Exception as e:
                print(f"An error occurred during forward pass: {e}")


In [80]:
# Step 5: Define a loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 6: Create DataLoader for batch training
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
print(f"y_train: {y_train.dtype}, {y_train.shape}")

y_train: torch.float32, torch.Size([211836, 1])


In [81]:
# Step 7: Train the model
train_model(model, train_loader, criterion, optimizer, device, num_epochs=10)

Inputs: torch.float32, torch.Size([64, 21])
Targets: torch.float32, torch.Size([64, 1])
An error occurred during forward pass: empty() received an invalid combination of arguments - got (Tensor, dtype=NoneType, device=NoneType), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)

Inputs: torch.float32, torch.Size([64, 21])
Targets: torch.float32, torch.Size([64, 1])
An error occurred during forward pass: empty() received an invalid combination of arguments - got (Tensor, dtype=NoneType, device=NoneType), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, 

KeyboardInterrupt: 

In [ ]:
# Step 8: Save the trained model parameters
torch.save(model.state_dict(), 'model.pth')